In [0]:
!pip install openpyxl

     |████████████████████████████████| 251 kB 5.5 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-022d3df0-e4fe-49bd-b94e-af19b3383f21/bin/python -m pip install --upgrade pip' command.


In [0]:
caminho_arquivo_bronze = "dbfs:/FileStore/stdevestudo/bronze/empresa/empresas.xlsx"

In [0]:
from pyspark.pandas import read_excel
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = read_excel(caminho_arquivo_bronze, dtype=str, skiprows=2)
df = df.to_spark()

/databricks/spark/python/pyspark/pandas/namespace.py:1126: FutureWarning: convert_float is deprecated and will be removed in a future version.
  return pd.read_excel(
/databricks/spark/python/pyspark/pandas/namespace.py:1126: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  return pd.read_excel(


In [0]:
display(df)

Unnamed: 0,Empresa A,Unnamed: 2,Unnamed: 3,Empresa B,Unnamed: 5,Unnamed: 6,Empresa C,Unnamed: 8,Unnamed: 9,Empresa D,Unnamed: 11,Unnamed: 12
null,Dado A,Dado B,Dado C,Dado A,Dado B,Dado C,Dado A,Dado B,Dado C,Dado A,Dado B,Dado C
2024-01-01 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200
2024-01-02 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200
2024-01-03 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200
2024-01-04 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200
2024-01-05 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200
2024-01-06 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200
2024-01-07 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200
2024-01-08 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200
2024-01-09 00:00:00,100,200,300,400,500,600,700,800,900,1000,1100,1200


In [0]:
# Definir o esquema
schema = StructType([
    StructField("data", DateType(), True),      # Coluna "data" do tipo DateType
    StructField("dado_a", IntegerType(), True), # Coluna "dado_a" do tipo IntegerType
    StructField("dado_b", IntegerType(), True), # Coluna "dado_b" do tipo IntegerType
    StructField("dado_c", IntegerType(), True), # Coluna "dado_c" do tipo IntegerType
    StructField("empresa", StringType(), True)  # Coluna "empresa" do tipo StringType
])

# Criar um DataFrame vazio com o esquema definido
df_full = spark.createDataFrame([], schema)

In [0]:
numeros = list(range(1, 13))

lista_indexs = []

for i in range(0, len(numeros), 3):
    lista_indexs.append([0] + numeros[i:i+3])

In [0]:
for index in lista_indexs:
    nomes_colunas = [df.columns[i] for i in index]

    df_aux = df.select(nomes_colunas)

    nome_empresa = df.columns[index[1]]
    print(nome_empresa)

    df_aux = df_aux.withColumnRenamed(nomes_colunas[0], "data")\
        .withColumnRenamed(nomes_colunas[1], "dado_a" )\
        .withColumnRenamed(nomes_colunas[2], "dado_b" )\
        .withColumnRenamed(nomes_colunas[3], "dado_c" )\
        .withColumn("empresa", lit(nome_empresa))

    df_aux = df_aux.filter(df_aux.data.isNotNull())

    df_full = df_full.union(df_aux)

Empresa A
Empresa B
Empresa C
Empresa D


In [0]:
df_full = df_full.select(
    "empresa",
    "data",
    "dado_a",
    "dado_b",
    "dado_c"
)

In [0]:
df_full = df_full.withColumn("empresa", col("empresa").cast(StringType()))\
                    .withColumn("data", date_format(df_full["data"], "yyyy-MM-dd"))\
                    .withColumn("data", to_date("data", "yyyy-MM-dd"))\
                    .withColumn("dado_a", col("dado_a").cast(IntegerType()))\
                    .withColumn("dado_b", col("dado_b").cast(IntegerType()))\
                    .withColumn("dado_c", col("dado_c").cast(IntegerType()))

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS empresa;

In [0]:
df_full.write.format("delta").mode("overwrite").saveAsTable("empresa.empresa")

In [0]:
%sql
SELECT * FROM empresa.empresa ORDER BY empresa;

empresa,data,dado_a,dado_b,dado_c
Empresa A,2024-01-01,100,200,300
Empresa A,2024-01-02,100,200,300
Empresa A,2024-01-03,100,200,300
Empresa A,2024-01-04,100,200,300
Empresa A,2024-01-05,100,200,300
Empresa A,2024-01-06,100,200,300
Empresa A,2024-01-07,100,200,300
Empresa A,2024-01-08,100,200,300
Empresa A,2024-01-09,100,200,300
Empresa A,2024-01-10,100,200,300
